# Transfer learning (uczenie transferowe)

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_380.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_517.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_364.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_258.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_237.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_476.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_563.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_159.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_464.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_518.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_141.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_398.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_442.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_443.jpg
/kaggl

In [3]:
from tensorflow.keras.applications.vgg19 import VGG19
?VGG19

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

Signature:
VGG19(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax',
)
Docstring:
Instantiates the VGG19 architecture.

Reference:
- [Very Deep Convolutional Networks for Large-Scale Image Recognition](
    https://arxiv.org/abs/1409.1556) (ICLR 2015)

For image classification use cases, see
[this page for detailed examples](
  https://keras.io/api/applications/#usage-examples-for-image-classification-models).

For transfer learning use cases, make sure to read the
[guide to transfer learning & fine-tuning](
  https://keras.io/guides/transfer_learning/).

The default input size for this model is 224x224.

Note: each Keras Application expects a specific kind of input preprocessing.
For VGG19, call `tf.keras.applications.vgg19.preprocess_input` on your
inputs before passing them to the model.
`vgg19.preprocess_input` will convert the input images from RGB to BGR,
then will zer

In [4]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.summary()

80134624/80134624 [==============================] - 0s 0us/step
Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [6]:
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))  # pierwszym parametrem 'output_shape' jest 'None'
top_model.add(Dense(128, activation='tanh'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

top_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 128)               3211392   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 3,211,521
Trainable params: 3,211,521
Non-trainable params: 0
_________________________________________________________________


In [7]:
model = Sequential()
model.add(base_model)
model.add(top_model)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 sequential (Sequential)     (None, 1)                 3211521   
                                                                 
Total params: 23,235,905
Trainable params: 23,235,905
Non-trainable params: 0
_________________________________________________________________


In [8]:
base_model.trainable = False  # wagi bazowego modelu nie będą się zmieniać

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 sequential (Sequential)     (None, 1)                 3211521   
                                                                 
Total params: 23,235,905
Trainable params: 3,211,521
Non-trainable params: 20,024,384
_________________________________________________________________


In [9]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Przygotowanie danych

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import preprocess_input  # wstępna obróbka obrazów tak, aby pasowały do struktury wytrenowanej sieci

In [17]:
train_data_dir = "/kaggle/input/cats-and-dogs-image-classification/train"

# tak się nie robi, ale tu wyjątkowo skorzystamy z części danych testowych do walidacji
val_data_dir = "/kaggle/input/cats-and-dogs-image-classification/test"
test_data_dir = "/kaggle/input/cats-and-dogs-image-classification/test"

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [18]:
?train_datagen.flow_from_directory

train_flow = train_datagen.flow_from_directory(train_data_dir,
                                               target_size=(224, 224),
                                               batch_size=32,
                                              class_mode="binary")

val_flow = val_datagen.flow_from_directory(val_data_dir,
                                               target_size=(224, 224),
                                               batch_size=32,
                                               class_mode="binary",
                                               shuffle=False)

test_flow = test_datagen.flow_from_directory(test_data_dir,
                                               target_size=(224, 224),
                                               batch_size=128,
                                               class_mode="binary",
                                               shuffle=False)

Found 557 images belonging to 2 classes.
Found 140 images belonging to 2 classes.
Found 140 images belonging to 2 classes.


Signature:
train_datagen.flow_from_directory(
    directory,
    target_size=(256, 256),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=None,
    save_to_dir=None,
    save_prefix='',
    save_format='png',
    follow_links=False,
    subset=None,
    interpolation='nearest',
    keep_aspect_ratio=False,
)
Docstring:
Takes the path to a directory & generates batches of augmented data.

Args:
    directory: string, path to the target directory. It should contain
      one subdirectory per class. Any PNG, JPG, BMP, PPM or TIF images
      inside each of the subdirectories directory tree will be included
      in the generator. See [this script](
      https://gist.github.com/fchollet/0830affa1f7f19fd47b06d4cf89ed44d)
      for more details.
    target_size: Tuple of integers `(height, width)`, defaults to `(256,
      256)`. The dimensions to which all images found will be resized.
    color_mode: One of "grayscale", "r

In [19]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(patience=3, restore_best_weights=True)

model.fit(train_flow, validation_data=val_flow, epochs=20, callbacks=[early_stopping])

Epoch 1/20
18/18 [==============================] - 6s 316ms/step - loss: 0.0547 - accuracy: 0.9749 - val_loss: 0.1654 - val_accuracy: 0.9357
Epoch 2/20
18/18 [==============================] - 6s 340ms/step - loss: 0.0457 - accuracy: 0.9856 - val_loss: 0.1469 - val_accuracy: 0.9357
Epoch 3/20
18/18 [==============================] - 6s 305ms/step - loss: 0.0342 - accuracy: 0.9928 - val_loss: 0.1440 - val_accuracy: 0.9286
Epoch 4/20
18/18 [==============================] - 6s 324ms/step - loss: 0.0295 - accuracy: 0.9910 - val_loss: 0.1363 - val_accuracy: 0.9429
Epoch 5/20
18/18 [==============================] - 6s 303ms/step - loss: 0.0367 - accuracy: 0.9892 - val_loss: 0.1252 - val_accuracy: 0.9500
Epoch 6/20
18/18 [==============================] - 6s 310ms/step - loss: 0.0301 - accuracy: 0.9928 - val_loss: 0.1198 - val_accuracy: 0.9500
Epoch 7/20
18/18 [==============================] - 6s 315ms/step - loss: 0.0277 - accuracy: 0.9928 - val_loss: 0.1167 - val_accuracy: 0.9500
Epoch 

## Augmentacja danych (obrazów)

In [20]:
?ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=preprocess_input
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)  # zbioru walidacyjnego nie zmieniamy, bo testujemy pewien wybór hiperparametrów

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

Init signature:
ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    interpolation_order=1,
    dtype=None,
)
Docstring:     
Generate batches of tensor image data with real-time data augmentation.

Deprecated: `tf.keras.preprocessing.image.ImageDataGenerator` is not
recommended for new code. Prefer loading images with
`tf.keras.utils.image_dataset_from_directory` and transforming the output
`tf.data.Dataset` with preprocessing layers. For more information, see the
tutoria

In [21]:
# zbiory danych są tak samo liczne, przekształcenia są wykonywane 'w locie' podczas każdej epoki sieci

train_flow = train_datagen.flow_from_directory(train_data_dir,
                                               target_size=(224, 224),
                                               batch_size=32,
                                              class_mode="binary")

val_flow = val_datagen.flow_from_directory(val_data_dir,
                                               target_size=(224, 224),
                                               batch_size=32,
                                               class_mode="binary",
                                               shuffle=False)

test_flow = test_datagen.flow_from_directory(test_data_dir,
                                               target_size=(224, 224),
                                               batch_size=128,
                                               class_mode="binary",
                                               shuffle=False)

Found 557 images belonging to 2 classes.
Found 140 images belonging to 2 classes.
Found 140 images belonging to 2 classes.


In [22]:
model.fit(train_flow, validation_data=val_flow, epochs=20, callbacks=[early_stopping])  # pogorszenie wydajności (prawdopodobnie wynika ze specyfiki danych "testowych")

Epoch 1/20
18/18 [==============================] - 11s 596ms/step - loss: 0.1083 - accuracy: 0.9569 - val_loss: 0.1644 - val_accuracy: 0.9286
Epoch 2/20
18/18 [==============================] - 11s 587ms/step - loss: 0.0832 - accuracy: 0.9677 - val_loss: 0.1644 - val_accuracy: 0.9214
Epoch 3/20
18/18 [==============================] - 11s 619ms/step - loss: 0.0943 - accuracy: 0.9641 - val_loss: 0.2593 - val_accuracy: 0.8857
Epoch 4/20
18/18 [==============================] - 11s 593ms/step - loss: 0.1070 - accuracy: 0.9515 - val_loss: 0.2087 - val_accuracy: 0.9071
Epoch 5/20
18/18 [==============================] - 11s 585ms/step - loss: 0.1156 - accuracy: 0.9569 - val_loss: 0.1560 - val_accuracy: 0.9214
Epoch 6/20
18/18 [==============================] - 11s 586ms/step - loss: 0.1129 - accuracy: 0.9569 - val_loss: 0.1572 - val_accuracy: 0.9143
Epoch 7/20
18/18 [==============================] - 11s 591ms/step - loss: 0.0926 - accuracy: 0.9641 - val_loss: 0.1642 - val_accuracy: 0.9071